# INNIO Fleet Analysis

In [2]:
import pandas as pd
import numpy as np
from IPython.display import HTML, display
import ipywidgets as widgets
import dmyplant2
dmyplant2.cred(); mp = dmyplant2.MyPlant(3600)

### Define Engines from installed fleet 

In [1]:
def sfun(x): return all([("Forsa Hartmoor" in str(x['IB Site Name'])),(x['OperationalCondition'] != 'Decommissioned')])

In [3]:
fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')
fleet.T

,0,1,2,3,4,5,6,7,8,9,10
Count_OpHour,188.0,185.0,172.0,165.0,180.0,185.0,170.0,157.0,158.0,121.0,157.0
OperationalCondition,Available,Available,Available,Available,Available,Available,Available,Available,Available,Available,Available
startup_counter,138.0,105.0,100.0,101.0,100.0,97.0,93.0,85.0,93.0,82.0,90.0
shutdown_counter,138.0,107.0,100.0,101.0,101.0,99.0,93.0,85.0,93.0,84.0,95.0
id,159396,159397,159399,159398,159400,159401,159403,159402,159404,159405,159406
Engine Version,H12,H12,H12,H12,H12,H12,H12,H12,H12,H12,H12
Engine Type,624,624,624,624,624,624,624,624,624,624,624
Engine Series,6,6,6,6,6,6,6,6,6,6,6
Country,GB,GB,GB,GB,GB,GB,GB,GB,GB,GB,GB
Commissioning Date,2022-01-10,2022-01-10,2022-01-11,2022-01-11,2022-01-22,2022-01-10,2022-01-11,2022-01-11,2022-01-11,2022-01-22,2022-01-11


In [ ]:
ddl = pd.DataFrame(fleet['serialNumber'] + ' - ' + fleet['IB Site Name'] + ' ' + fleet['Engine ID'])[0].to_list()
ddl = [(m, i) for i, m in enumerate(ddl)]
dl=widgets.Dropdown(options = ddl,value=0,description='Engine:',layout={'width':'max-content'},disabled=False)
display(dl)

In [ ]:
pd.DataFrame(fsm._starts)[['index'] +['startpreparation','starter','hochlauf','idle','synchronize','loadramp']].round(2)

In [ ]:
rmodes = ['???','OFF','MANUAL','AUTO']; mw = [] ; modes = []
for mm in rmodes:
    mw.append(widgets.Checkbox(value=False, description='Mode: ' + mm))
display(widgets.VBox(mw))

In [ ]:
rsucc = [True,False]; sw = []; success=[]
for rs in rsucc:
    sw.append(widgets.Checkbox(value=False, description='Success: ' + str(rs)))
display(widgets.VBox(sw))

In [ ]:
import matplotlib.pyplot as plt
import bokeh
import arrow
from collections import namedtuple

from pprint import (pprint as pp, pformat as pf)

import warnings
pd.options.mode.chained_assignment = None # default warn => SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=UserWarning)